In [65]:
%load_ext autoreload
%autoreload 2
from slack import SlackInteractor
from llm import LLMInteractor

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
interactor = SlackInteractor()
data = interactor.fetch_new_messages()

Set conversations_oldest to 1722399936.894109
Loaded 35 old messages
Found 0 new messages
Updated complete_conversations.pkl with new messages


In [67]:
threads = interactor.organize_threads(data)

In [68]:
interactor.get_conversation_stats(data)

,max_len,total_length,total_messages,average_message_size
user_name,,,,


In [74]:
from pandas import Timestamp

llm_interactor = LLMInteractor()
thread = {'channel': 'general',
  'thread_ts': Timestamp('2024-08-01 23:18:53.991878986'),
  'messages': [{'ts': Timestamp('2024-08-01 23:18:53.991878986'),
    'user': 'Eli.chen',
    'text': 'Can you let me know what you think of SF for our offsite everyone?',
    'is_bot': False,
    'minutes_ago': 153}]}
response = llm_interactor.review_and_remind(thread)
response

{'new_items': ['Here is my analysis of the conversation for any new tasks, action items, or requests:',
  '',
  'New action items:',
  "1. Provide opinion on holding offsite meeting in San Francisco (SF): The human has requested your thoughts on whether San Francisco would be a good location for the group's offsite meeting."],
 'reminders': "Hi there! I hope you're having a great day. I just wanted to follow up on our conversation about the potential offsite meeting location. When you have a moment, I'd love to hear your thoughts on San Francisco as an option. Your input is really valuable in making this decision. Please don't hesitate to share any ideas or insights you might have. Looking forward to discussing further whenever works best for you. Thanks!"}

In [58]:
for thread in threads:
    if not thread['messages'][-1]['is_bot']:
        response = llm_interactor.generate_thread_response(thread, guideline_prompt)
        interactor.post_thread_reply(thread, response)

Posted reply to thread 1722632063.551869 in channel general
Posted reply to thread 1722632067.855989 in channel general
